In [15]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

img_height =255
img_width =255
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
    )


In [16]:
num = 20

def prepare(ds, batch = 1, shuffle=False, augment=False):
    preprocess_input = tf.keras.applications.mobilenet_v3.preprocess_input
    ds = ds.map(lambda x,y : (tf.image.resize(x,[img_height, img_width]), y),
                num_parallel_calls=AUTOTUNE)
    #batch all datasets
    ds = ds.batch(batch_size)
        
    return ds.prefetch(buffer_size =AUTOTUNE)

In [17]:
num_classes = metadata.features['label'].num_classes
label_name = metadata.features['label'].names
print(label_name, ", classnum :", num_classes, ", type: ", type(label_name))

test_ds = prepare(test_ds, num)
image_test, label_test = next(iter(test_ds))
image_test = np.array(image_test)
label_test = np.array(label_test, dtype='int')

#모델 불러오기
model = tf.keras.models.load_model('transfer_learning_flower.keras')
model.summary()

predict = model.predict(image_test)
predicted_classes = np.argmax(predict, axis=1)



['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses'] , classnum : 5 , type:  <class 'list'>


2025-06-24 11:58:46.146533: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 255, 255, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MobileNetV3Small (Functional)   │ (None, 8, 8, 576)      │       939,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 576)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         2,885 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 947,777 (3.62 MB)

 Trainable params: 2,885 (11.27 KB)

 Non-trainable params: 939,120 (3.58 MB)

 Optimizer params: 5,772 (22.55 KB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step


In [18]:
print("실제 레이블 | 예측 레이블")
print("--------------------")

for ll in range((label_test.size)):
    print(label_name[label_test[ll]], "|",
          label_name[predicted_classes[ll]])
    print("-------------------------------")
    
accuracy = np.mean(predicted_classes == label_test)
print(f"정확도:{accuracy:.2%}")

실제 레이블 | 예측 레이블
--------------------
roses | tulips
-------------------------------
dandelion | dandelion
-------------------------------
dandelion | dandelion
-------------------------------
tulips | tulips
-------------------------------
dandelion | dandelion
-------------------------------
dandelion | dandelion
-------------------------------
tulips | tulips
-------------------------------
daisy | daisy
-------------------------------
sunflowers | sunflowers
-------------------------------
dandelion | dandelion
-------------------------------
dandelion | dandelion
-------------------------------
dandelion | sunflowers
-------------------------------
sunflowers | sunflowers
-------------------------------
roses | tulips
-------------------------------
dandelion | dandelion
-------------------------------
sunflowers | sunflowers
-------------------------------
tulips | tulips
-------------------------------
dandelion | dandelion
-------------------------------
tulips | tulips
--------